In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim

True


In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307, ), (0.3081, ))])

train_data = datasets.MNIST(root="./dataset", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./dataset", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


In [ ]:
model = Net()

criterion = torch.nn.CrossEntropyLoss()
optim = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    running_loss = 0.0
    for batch_inx, data in enumerate(train_loader, 0):
        input, target = data
        optim.zero_grad()

        outputs = model(input)
        loss = criterion(outputs, target)
        loss.backward()
        optim.step()

        running_loss += loss.item()
        if batch_inx % 300 == 299:
            print("[%d, %5d] loss: %.3f" %(epoch + 1, batch_inx + 1, running_loss / 300))
            running_loss = 0

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_data:
            images, label = data
            outputs = model(images)
            _, predict = torch.max(outputs.data, dim=1)
            total += label.size(0)
            correct += (predict == label).sum().item()
    print("Accuracy on test set: %d %d" %(100 * correct / total))

In [ ]:
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()